In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели
df = pd.read_csv('main_task.csv')
df['Number of Reviews']=df['Number of Reviews'].fillna(df['Number of Reviews'].mean())
#кол-во кухонь в каждом ресторане
pat = re.compile('\w+\s*\w*\s*\w*')
d = df['Cuisine Style'].apply(lambda x: ['default'] if pd.isnull(x) else pat.findall(x))
df['Count_cuisine'] = d.apply(lambda x: len(x))

In [3]:
#наличие отзывов,если есть отзывы в поле Reviews, тогда 1 , иначе 0
pats = re.compile('\w+\s*\w*\s*\w*')
df['Have_review'] = df['Reviews'].apply(lambda x: ['default'] if pd.isnull(x) else pats.findall(x))
df['Have_review']=df['Have_review'].str[0].replace(pats, 1)
df['Have_review']=df['Have_review'].fillna(0) 

In [4]:
#замена уровня среднего чека
df['Price Range']=df['Price Range'].fillna('$')
Price_dict = {'$': 1,'$$ - $$$': 2,'$$$$': 3}
df['Price Range'] = df['Price Range'].replace(to_replace=Price_dict)

In [5]:
#кол-во дней между отзывами
paty = re.compile('[0-9]{2}[/]{1}[0-9]{2}[/]{1}[0-9]{4}')
df['Days'] = df['Reviews'].dropna().apply(lambda x: ['default'] if pd.isnull(x) else paty.findall(x))
df['Days']= pd.to_datetime(df['Days'].str[1])-pd.to_datetime(df['Days'].str[0])
df['Days']= df['Days'].dt.days
df['Days']= df['Days'].fillna(0)

In [14]:
#Оценка тональности
#создаю словари со словами, которые могут охарактерировать отзыв
great = ["Delicious", "Great", "tasteful","Awesome", "Must Visit", "Brilliant","delicious", "great","favourites", "Recommend it", "EXCELLENT", "Perfect", "tasty", "COMPLETELY REAL", "outstanding", "Favourite", "Wonderful", "fantastic", "Fantastic", "Superb", "Amazing",
         "BEST", "wonderful","Nice","Unique ", "perfect", "Best", "best", "AMAZING","Amazing", "favorite", "Excellent", "excellent", "GREAT", "Cool", "cool", "Delightful"]
good = ["Good", "good", "average","Pleasant", "surprise","hospitable", "Cosy", "Soso", "Average", "Beautiful", "nice", "No bad", "Friendly", "friendly", "charm",
        "no bad", "Not bad", "pleasant","Welcoming", "is ok", "OK", "not bad", "relaxing", "Relaxing", "Alright", "Love", "Tasty"]
bad = ["No", "no", "Mediocre", "not", "Not", "poor", "bad", "Worst", "Bad", "Shameful",
       "Too busy", "appalling", "rude", "Dirty", "Underwhelmed","Awful", "hidden gem", "Horrible", "horrible", "Disappointed", "Don't eat", "Avoid", "could improve", "Dissapointed"]
pats = re.compile('\w+\s*\w*\s*\w*')
df['Reviews_new'] = df['Reviews'].apply(lambda x: ['default'] if pd.isnull(x) else pats.findall(x))

def setMarksToReview(s):
    mark = 0    
    for st in s:
        for s in st.split():
            if s in great:
                mark = 5
                break
            elif s in bad:
                mark = 0
                break
            elif s in good:
                mark = 3.5
                break 
    else: 
        return (mark)
    
df['ReviewsMark'] = df['Reviews_new'].apply(lambda x: setMarksToReview(x))


In [ ]:
# get_dummies для города
df = pd.get_dummies(df, columns=[ 'City',], dummy_na=True)

In [15]:
df

,Restaurant_id,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,Count_cuisine,...,City_Oporto,City_Oslo,City_Paris,City_Prague,City_Rome,City_Stockholm,City_Vienna,City_Warsaw,City_Zurich,City_nan
0,id_5569,"['European', 'French', 'International']",5570.0,3.5,2,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,3,...,0,0,1,0,0,0,0,0,0,0
1,id_1535,NaN,1537.0,4.0,1,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,1,...,0,0,0,0,0,1,0,0,0,0
2,id_352,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,7,...,0,0,0,0,0,0,0,0,0,0
3,id_3456,NaN,3458.0,5.0,1,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,1,...,0,0,0,0,0,0,0,0,0,0
4,id_615,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,2,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,4,...,0,0,0,0,0,0,0,0,0,0
39996,id_6340,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,2,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,5,...,0,0,1,0,0,0,0,0,0,0
39997,id_1649,"['Japanese', 'Sushi']",1652.0,4.5,1,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,2,...,0,0,0,0,0,1,0,0,0,0
39998,id_640,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,2,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,5,...,0,0,0,0,0,0,0,1,0,0


In [18]:
object_columns = [s for s in df.columns if df[s].dtypes == 'object']
df.drop(object_columns, axis = 1, inplace=True)
df

,Ranking,Rating,Price Range,Number of Reviews,Count_cuisine,Have_review,Days,ReviewsMark,City_Amsterdam,City_Athens,...,City_Oporto,City_Oslo,City_Paris,City_Prague,City_Rome,City_Stockholm,City_Vienna,City_Warsaw,City_Zurich,City_nan
0,5570.0,3.5,2,194.0,3,1.0,-41.0,3.5,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1537.0,4.0,1,10.0,1,1.0,-382.0,5.0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,353.0,4.5,3,688.0,7,1.0,-2.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3458.0,5.0,1,3.0,1,0.0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,621.0,4.0,2,84.0,3,1.0,-272.0,5.0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,500.0,4.5,2,79.0,4,1.0,-34.0,5.0,0,0,...,0,0,0,0,0,0,0,0,0,0
39996,6341.0,3.5,2,542.0,5,1.0,-9.0,3.5,0,0,...,0,0,1,0,0,0,0,0,0,0
39997,1652.0,4.5,1,4.0,2,1.0,-3127.0,3.5,0,0,...,0,0,0,0,0,1,0,0,0,0
39998,641.0,4.0,2,70.0,5,1.0,-23.0,0.0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [21]:
X = df.drop('Rating', axis = 1)
y = df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21522199999999997
